## Assignment 4

In Assignment 4 of Models for Language Prodcessing, you are asked to train and test linear and logistic regression models and work with pretrained language models.

# <font color="red">Contributions</font>


* 42
* Thijmen van der Meijden, Viggo de Breij, Lex Klaassen
* Up until 4.4 all contributed, 4.5 Thijmen, 4.6 Thijmen, 4.7 all contributed, 4.8 -.

To start the assignment, import prerequisite packages:

In [1]:
import torch
import torch.nn.functional as F
import torchtext as text
import numpy as np
from tqdm.notebook import trange, tqdm
import nltk,sklearn
from sklearn.model_selection import train_test_split

In [2]:
import pandas as pd
import collections, itertools
import more_itertools

# 4.1 (2 points) Import GloVe word embedding model

GloVe contains _static_ word embeddings. This means that the vector is assigned to word types and does not vary in different contexts or for different word senses. Pretrained GloVe word embedding models exist in different sizes. For the purpose of the exercise, we will use the smallest GloVe vectors with 50 dimensions. First, let's download the vectors. This may take a few minutes.

In [3]:
glovedim=50
vec = text.vocab.GloVe(name='6B', dim=glovedim)

.vector_cache/glove.6B.zip: 862MB [02:39, 5.39MB/s]                           
100%|█████████▉| 399999/400000 [00:13<00:00, 29980.33it/s]


How many words does it contain?

In [4]:
print(f"there are {len(vec.itos)} vectors, and each vector encodes a word, so there are that much words in the vocab")

there are 400000 vectors, and each vector encodes a word, so there are that much words in the vocab


Check if a word has a GloVe embedding. For example, _cat_ has a vector but
_cact_ does not. Check for the inclusion of these words:

In [5]:
#check if "cat" has a GloVe vector, return a Boolean value
"cat" in vec.itos

True

In [6]:
#check if "cact" has a GloVe vector, return a Boolean value
"cact" in vec.itos

False

What is the vector of _cat_?



In [7]:
vec["cat"]

tensor([ 0.4528, -0.5011, -0.5371, -0.0157,  0.2219,  0.5460, -0.6730, -0.6891,
         0.6349, -0.1973,  0.3368,  0.7735,  0.9009,  0.3849,  0.3837,  0.2657,
        -0.0806,  0.6109, -1.2894, -0.2231, -0.6158,  0.2170,  0.3561,  0.4450,
         0.6089, -1.1633, -1.1579,  0.3612,  0.1047, -0.7832,  1.4352,  0.1863,
        -0.2611,  0.8328, -0.2312,  0.3248,  0.1449, -0.4455,  0.3350, -0.9595,
        -0.0975,  0.4814, -0.4335,  0.6945,  0.9104, -0.2817,  0.4164, -1.2609,
         0.7128,  0.2378])

# (4 points) 4.2 Linear regression: Concreteness prediction

Obtain concreteness ratings from the paper

Brysbaert, M., Warriner, A., & Kuperman, V. (2014). Concreteness ratings for 40 thousand generally known English word lemmas. BEHAVIOR RESEARCH METHODS, 46 (3), 904–911. https://doi.org/10.3758/s13428-013-0403-5

In [8]:
!wget http://crr.ugent.be/papers/Concreteness_ratings_Brysbaert_et_al_BRM.txt

--2023-06-15 21:14:21--  http://crr.ugent.be/papers/Concreteness_ratings_Brysbaert_et_al_BRM.txt
Resolving crr.ugent.be (crr.ugent.be)... 157.193.153.14
Connecting to crr.ugent.be (crr.ugent.be)|157.193.153.14|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1646191 (1.6M) [text/plain]
Saving to: ‘Concreteness_ratings_Brysbaert_et_al_BRM.txt’

Concreteness_rating 100%[===================>]   1.57M   799KB/s    in 2.0s    

2023-06-15 21:14:23 (799 KB/s) - ‘Concreteness_ratings_Brysbaert_et_al_BRM.txt’ saved [1646191/1646191]



This is a tab-separated file with a header. Structured data like this can be conveniently read via DictReader class from csv.

In [9]:
import csv

Using DictReader, create lists ```concreteness_words``` and ```concreteness_scores``` of words in the concreteness ratings file that have a GloVe vector

In [10]:
concreteness_words=[]
concreteness_scores=[]
with open("Concreteness_ratings_Brysbaert_et_al_BRM.txt",'r') as concfile:
  read_tsv = csv.DictReader(concfile, delimiter="\t")
  # Our code below:
  word_set = set(vec.itos)
  for bieb in read_tsv:
    if bieb["Word"] in word_set:
      concreteness_words.append(bieb["Word"])
      concreteness_scores.append(float(bieb["Conc.M"]))


How many words ended up in your concreteness dataset?

In [11]:
len(concreteness_words)

31618

Create train and test partitions of the concreteness data:

In [12]:
conc_words_train, conc_words_test, conc_scores_train, conc_scores_test = train_test_split(concreteness_words,concreteness_scores,test_size=0.1)

Convert data to torch tensor format to use in a regression model:

In [14]:
#here we *stack* vectors for all words in a single torch tensor:
vecs_train=torch.stack([vec[w] for w in conc_words_train])
vecs_test=torch.stack([vec[w] for w in conc_words_test])
#here we convert lists of scores into a tensor
scores_train=torch.tensor(conc_scores_train)
scores_test=torch.tensor(conc_scores_test)

Now we can define linear regression model in pyTorch:

In [15]:
class Regression(torch.nn.Module):
     def __init__(self, input_dim, output_dim):
         super(Regression, self).__init__()
         self.linear = torch.nn.Linear(input_dim, output_dim)
     def forward(self, x):
         outputs = self.linear(x)
         return outputs

A specific linear regression model can have the input dimensionality of our word embeddings and 1-dimensional input (the concretenss score)

In [16]:
model = Regression(glovedim,1)

To train the model, we need a loss function and an optimiser:

In [17]:
criterion = torch.nn.MSELoss()

In [18]:
optimizer = torch.optim.SGD(model.parameters(), lr=0.1)

We can now train our linear regression model using gradient descent. Every 5 training epochs, evaluate the model, printing out loss and accuracy for the training and test sets. Calculate the accuracy as the percentage of examples where the predicted score of the model differs from the correct score by less than 1. The training will take less than a minute.

In [19]:
losses = []
losses_test = []
Iterations = []
epochs = 20
tolerance = 1.0
i = 0

#after defining parameters, we train the model several times on the same data; each iteration is an epoch:
for epoch in trange(epochs, desc='Training Epochs'):
    x = vecs_train
    optimizer.zero_grad()
    #here we pass the word vectors from the training set, obtaining regression model's predicted outputs:
    outputs = model(x)
    loss = criterion(torch.squeeze(outputs), scores_train)
    loss.backward()
    optimizer.step()
    i += 1

    #Now, every 5 epochs we can evaluate how the model performs on the data
    if (epoch + 1) % 5 == 0:
        #we don't compute gradients as the model is only evaluated and not updated
        with torch.no_grad():
            model.eval()
            #complete the code for evaluating the model here
            tellertje = 0  #counter for correct answers in training and testdata
            tellertjetest = 0

            outputstrain = model(vecs_train)
            outputstest = model(vecs_test)

            for k, score_train in enumerate(scores_train):
                if abs(score_train - outputs[k]) < tolerance:
                    tellertje += 1
            for l, score_test in enumerate(scores_test):
                if abs(score_test - outputstest[l]) < tolerance:
                    tellertjetest +=1


            print(f'Training accuracy: \t{round(tellertje / len(scores_train), 4)*100}% \t \t \t Epoch: {i}')         #correct answers divided by total, %accuracy
            print(f'Training loss: \t \t{round(float(criterion(torch.squeeze(outputstrain), scores_train)), 2)}')
            print(f'Test accuracy: \t \t{round(tellertjetest / len(scores_test), 4)*100}%')
            print(f'Test loss: \t \t{round(float(criterion(torch.squeeze(outputstest), scores_test)), 2)} \n')

Training Epochs:   0%|          | 0/20 [00:00<?, ?it/s]

Training accuracy: 	59.47% 	 	 	 Epoch: 5
Training loss: 	 	1.45
Test accuracy: 	 	61.99%
Test loss: 	 	1.43 

Training accuracy: 	68.04% 	 	 	 Epoch: 10
Training loss: 	 	1.04
Test accuracy: 	 	69.86%
Test loss: 	 	1.02 

Training accuracy: 	72.88% 	 	 	 Epoch: 15
Training loss: 	 	0.85
Test accuracy: 	 	74.45%
Test loss: 	 	0.84 

Training accuracy: 	75.78% 	 	 	 Epoch: 20
Training loss: 	 	0.73
Test accuracy: 	 	77.64%
Test loss: 	 	0.72 



What is the predicted concreteness score of the noun _abstractness_?

In [25]:
round(float(model(vec["abstractness"]).detach()), 4)

1.9904

What is the predicted concreteness score of the noun _dog_?

In [24]:
round(float(model(vec["dog"]).detach()), 4)

5.7853

# 4.3. (1 point) Create a dataset of WordNet supersenses for words that have GloVe vectors.

First, download the WordNet database:

In [26]:
nltk.download('wordnet')
from nltk.corpus import wordnet as wn

[nltk_data] Downloading package wordnet to /root/nltk_data...


Now, create a dataset that includes for each word in WordNet that has a GloVe vector the lexicographer file (also known as lexical file or supersense) of its first synset. Lexicographer fine information is available as an attribute of the synset. If you browse WordNet online, you can select the option "Show Lexical File Info'' to see which supersenses words have.

See WordNet documentation for the full list of supersenses: https://wordnet.princeton.edu/documentation/lexnames5wn

In [27]:
vec_itos_set = set(vec.itos)
wn_words = [word for word in list(wn.words()) if word in vec_itos_set]

wn_supersenses_name = [wn.synsets(word)[0].lexname() for word in wn_words]
supsens_set = set(wn_supersenses_name)
wn_supersenses = [list(supsens_set).index(s) for s in wn_supersenses_name]

Split the dataset into train and test partitions:

In [28]:
wn_words_train, wn_words_test, wn_supersenses_train, wn_supersenses_test = train_test_split(wn_words,wn_supersenses,test_size=0.1)

# 4.4. (5 points) Logistic regression: word class prediction.

Now we can address a classification task. Define a (multinomial) regression model using softmax, choose a loss function and an optimizer for it.

In [29]:
num_classes = len(set(wn_supersenses))

Initialize your model. Use the same Regression class for logistic regression as for linear regression - the difference will come from the objective (loss) function.

In [38]:
logreg_model = Regression(glovedim,num_classes)

Choose the loss function. This is a crucial choice: some of the loss functions in PyTorch (see https://pytorch.org/docs/stable/nn.html) already include a softmax or a sigmoid in their implementaion, which gives computational advantages. Be sure to read the documentation on your loss function to confirm you made a good choice.

In [39]:
loss_function = torch.nn.CrossEntropyLoss()

And initialize the optimiser:

In [40]:
logregoptimiser=torch.optim.Adam(logreg_model.parameters())

WordNet is quite big. For efficiency, use the following function for splitting your data into batches when processing:

In [41]:
batch_size = 500
def get_batches(src_iter, tgt_iter, batch_size=batch_size):
    for batch in more_itertools.chunked(zip(src_iter, tgt_iter), batch_size):
        x, y = zip(*batch)
        x = torch.stack(x)
        y = torch.stack(y)
        yield x, y

Train and test your logistic regression model, printing the train and test loss and accuracy:

In [42]:
batch = 0
epochs = 20

vec_words_train = torch.stack([vec[word] for word in wn_words_train])
vec_words_test = torch.stack([vec[word] for word in wn_words_test])

for epoch in trange(epochs, desc='Training Epochs'):
    logreg_model.train()  # Set the model to training mode

    x = vec_words_train
    y = torch.tensor(wn_supersenses_train)

    for x_batch, y_batch in get_batches(x, y, batch_size):
        batch += 1
        logregoptimiser.zero_grad()
        outputs = logreg_model(x_batch)
        loss = loss_function(torch.squeeze(outputs), y_batch)
        loss.backward()
        logregoptimiser.step()

with torch.no_grad():
    logreg_model.eval()
    tellertje_train = 0
    tellertje_test = 0

    outputs_train = logreg_model(vec_words_train)
    y_train = torch.tensor(wn_supersenses_train)

    outputs_test = logreg_model(vec_words_test)
    y_test = torch.tensor(wn_supersenses_test)

    for i, score in enumerate(outputs_train):
      predicted_index = torch.argmax(score)
      if predicted_index == y_train[i]:
        tellertje_train += 1

    for i, score in enumerate(outputs_test):
      predicted_index = torch.argmax(score)
      if predicted_index == y_test[i]:
        tellertje_test += 1

    print(f'Training accuracy: \t{round(tellertje_train/len(wn_words_train), 4)*100}%')         #correct answers divided by total, %accuracy
    print(f'Training loss: \t \t{round(float(loss_function(torch.squeeze(outputs_train), y_train)), 2)}')
    print(f'Test accuracy: \t \t{round(tellertje_test/len(wn_words_test), 4)*100}%')
    print(f'Test loss: \t \t{round(float(loss_function(torch.squeeze(outputs_test), y_test)), 2)} \n')

Training Epochs:   0%|          | 0/20 [00:00<?, ?it/s]

Training accuracy: 	46.93%
Training loss: 	 	1.93
Test accuracy: 	 	46.61%
Test loss: 	 	1.97 



Define a dictionary mapping from indices to lexicographic file names:

In [43]:
itolexname = {int(supersense): wn_supersenses_name[i] for i, supersense in enumerate(wn_supersenses)}

In [45]:
itolexname[2] # It is not 1 to 1 translateble to the lexical file numbers of the documentation because it assigns random indexes to the names.

'noun.shape'

Which supersense (lexicographic file) does your classifier assign to the noun _abstractness_?

In [46]:
#your code
lijst = list(float(element) for element in logreg_model(vec["abstractness"]).detach())
print(itolexname[lijst.index(max(lijst))])

adj.all


Which supersense (lexicographic file) does your classifier assign to the noun _dog_?

In [47]:
#your code
lijst = list(float(element) for element in logreg_model(vec["dog"]).detach())
print(itolexname[lijst.index(max(lijst))])

noun.animal


# 4.5. (5 points) Hypernymy classification

Now, download a lexical entailment (hypernymy) dataset called WBLESS. The dataset was developed by Weeds et al. with the goal of testing models on distinguishing hypernyms from other related word pairs.

Weeds et al. (2014) Julie Weeds, Daoud Clarke, Jeremy Reffin, David Weir, and Bill Keller. 2014. Learning to distinguish hypernyms and co-hyponyms. In Proceedings of the 2014 International Conference on Computational Linguistics, pages 2249–2259, Dublin, Ireland.

WBLESS (together with other relevant datasets) can be conveniently downloaded from the Facebook Research github page by Stephen Roller who worked on hypernymy learning:

Stephen Roller, Douwe Kiela, and Maximilian Nickel. 2018. Hearst Patterns Revisited: Automatic Hypernym Detection from Large Text Corpora. ACL.

Download the tab-separated dataset:

In [48]:
!wget https://github.com/facebookresearch/hypernymysuite/raw/main/data/wbless.tsv

--2023-06-15 21:21:26--  https://github.com/facebookresearch/hypernymysuite/raw/main/data/wbless.tsv
Resolving github.com (github.com)... 140.82.113.4
Connecting to github.com (github.com)|140.82.113.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/facebookresearch/hypernymysuite/main/data/wbless.tsv [following]
--2023-06-15 21:21:26--  https://raw.githubusercontent.com/facebookresearch/hypernymysuite/main/data/wbless.tsv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 51267 (50K) [text/plain]
Saving to: ‘wbless.tsv’

wbless.tsv          100%[===================>]  50.07K  --.-KB/s    in 0.01s   

2023-06-15 21:21:27 (4.55 MB/s) - ‘wbless.tsv’ saved [51267/51267]



Check how the data looks:

In [49]:
!head wbless.tsv

word1	word2	label	relation	fold
frigate	craft	True	hyper	val
trouble	carp	False	other	val
fox	mouth	False	other	val
foot	robin	False	other	val
vest	garment	True	hyper	val
beetle	insect	True	hyper	val
axe	utensil	True	hyper	val
chair	scroll	False	other	val
end	spear	False	other	val


We used ```csv``` above to process a tab separated file. It can also be done using ```pandas```:

In [50]:
wbless_df = pd.read_csv('wbless.tsv', sep='\t')
wbless_df.head()

,word1,word2,label,relation,fold
0,frigate,craft,True,hyper,val
1,trouble,carp,False,other,val
2,fox,mouth,False,other,val
3,foot,robin,False,other,val
4,vest,garment,True,hyper,val


Now create training and test data for relation classification:

In [51]:
wbless_words = wbless_df[['word1', 'word2']]
hypernymy = wbless_df['relation']

Split into training and test data:

In [52]:
wbless_words_train, wbless_words_test, hypernymy_train, hypernymy_test = train_test_split(wbless_words,hypernymy,test_size=0.1)

Finally, create, train and test a logistic regression model that predicts whether two words stand in the hypernymy relation given their GloVe vectors. Print the train and test loss and accuracy. Use the concatenation of the two words' vectors as input to the logistic regression classifier.

First, prepare your data by converting it into the tensor format:

In [58]:
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

vecpairs_train = torch.tensor([np.concatenate([vec[word1.lower()], vec[word2.lower()]]) for word1, word2 in wbless_words_train.values])
vecpairs_test = torch.tensor([np.concatenate([vec[word1.lower()], vec[word2.lower()]]) for word1, word2 in wbless_words_test.values])
labels_train = torch.tensor([float(1) if label == 'hyper' else float(0) for label in hypernymy_train])
labels_test = torch.tensor([float(1) if label == 'hyper' else float(0) for label in hypernymy_test])

Initialize the model:

In [59]:
logreg_model = Regression(2*glovedim,1)

Choose the loss function and train the model for 20 epochs. Note that we are dealing here with binary classification (predicting probability of hypernymy), not multiple class logistic regression as in he exercise before. This has consequences for the choices you make for your code.

In [60]:
epochs = 20

m = torch.nn.Sigmoid()
loss_function = torch.nn.BCEWithLogitsLoss()
logregoptimiser=torch.optim.Adam(logreg_model.parameters(), lr=0.05)

for epoch in trange(epochs, desc='Training Epochs'):
    logreg_model.train()  # Set the model to training mode

    x_train = vecpairs_train
    y_train = labels_train

    x_test = vecpairs_test
    y_test = labels_test

    logregoptimiser.zero_grad()
    outputs = logreg_model(x_train)
    loss = loss_function(torch.squeeze(outputs), y_train)
    loss.backward()
    logregoptimiser.step()

with torch.no_grad():
    logreg_model.eval()

    threshold = 0.5
    tellertje_train = 0
    tellertje_test = 0

    outputs_train = logreg_model(x_train)
    outputs_test = logreg_model(x_test)

    for i, value in enumerate(outputs_train):
      predicted_index = int(value >= threshold)
      if predicted_index == int(y_train[i]):
        tellertje_train += 1

    for i, value in enumerate(outputs_test):
      predicted_index = int(value >= threshold)
      if predicted_index == int(y_test[i]):
        tellertje_test += 1

    print(f'Training accuracy: \t{round(tellertje_train/len(vecpairs_train), 4)*100}%')         #correct answers divided by total, %accuracy
    print(f'Training loss: \t \t{round(float(loss_function(torch.squeeze(outputs_train), y_train)), 2)}')
    print(f'Test accuracy: \t \t{round(tellertje_test/len(vecpairs_test), 4)*100}%')
    print(f'Test loss: \t \t{round(float(loss_function(torch.squeeze(outputs_test), y_test)), 2)} \n')


Training Epochs:   0%|          | 0/20 [00:00<?, ?it/s]

Training accuracy: 	90.67%
Training loss: 	 	0.24
Test accuracy: 	 	90.42%
Test loss: 	 	0.21 



What label does your model predict for the pair _dog,animal_? Hint: your code below should produce a tensor of size ```torch.Size([1])``` containing a single Boolean value.

In [61]:
predicted_label = int(logreg_model(torch.tensor(np.concatenate([vec["dog"], vec["animal"]]))) >= 0.5)

print("Predicted Label:", predicted_label)

Predicted Label: 1


What label does your model predict for the pair _dog,cat_?

In [62]:
predicted_label = int(logreg_model(torch.tensor(np.concatenate([vec["dog"], vec["cat"]]))) >= 0.5)

print("Predicted Label:", predicted_label)

Predicted Label: 0


What label does your model predict for the pair _animal,dog_?

In [63]:
predicted_label = int(logreg_model(torch.tensor(np.concatenate([vec["animal"], vec["dog"]]))) >= 0.5)

print("Predicted Label:", predicted_label)

Predicted Label: 0


#4.6 (1 point) Read the SICK dataset

Above, we did entailment (hypernymy) classification on the basis of word vectors. Now we can do a similar experiment for sentence embeddings. Start by downloading the SICK dataset:

In [64]:
!wget https://zenodo.org/record/2787612/files/SICK.zip?download=1 -O SICK.zip
!unzip SICK.zip
!rm SICK.zip

--2023-06-15 21:24:44--  https://zenodo.org/record/2787612/files/SICK.zip?download=1
Resolving zenodo.org (zenodo.org)... 188.185.124.72
Connecting to zenodo.org (zenodo.org)|188.185.124.72|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 217584 (212K) [application/octet-stream]
Saving to: ‘SICK.zip’

SICK.zip            100%[===================>] 212.48K   656KB/s    in 0.3s    

2023-06-15 21:24:46 (656 KB/s) - ‘SICK.zip’ saved [217584/217584]

Archive:  SICK.zip
  inflating: readme.txt              
  inflating: SICK.txt                


In [65]:
!head SICK.txt

pair_ID	sentence_A	sentence_B	entailment_label	relatedness_score	entailment_AB	entailment_BA	sentence_A_original	sentence_B_original	sentence_A_dataset	sentence_B_dataset	SemEval_set
1	A group of kids is playing in a yard and an old man is standing in the background	A group of boys in a yard is playing and a man is standing in the background	NEUTRAL	4.5	A_neutral_B	B_neutral_A	A group of children playing in a yard, a man in the background.	A group of children playing in a yard, a man in the background.	FLICKR	FLICKR	TRAIN
2	A group of children is playing in the house and there is no man standing in the background	A group of kids is playing in a yard and an old man is standing in the background	NEUTRAL	3.2	A_contradicts_B	B_neutral_A	A group of children playing in a yard, a man in the background.	A group of children playing in a yard, a man in the background.	FLICKR	FLICKR	TRAIN
3	The young boys are playing outdoors and the man is smiling nearby	The kids are playing outdoors near a man 

Read the train and test data from the file. Be sure to include in the training data all sentence pairs marked as "train" or "trial" in the SICK.txt file, and in the test data all sentence pairs marked as "test". Be sure to read the SICK examples into lists of pairs of sentence strings:

In [66]:
sick_df = pd.read_csv('SICK.txt', sep='\t')

sick_train_df = sick_df[(sick_df['SemEval_set'] == 'TRAIN') | (sick_df['SemEval_set'] == 'TRIAL')]
sick_train_examples = list(zip(sick_train_df['sentence_A'].tolist(), sick_train_df['sentence_B'].tolist()))
sick_train_labels = sick_train_df['entailment_label'].tolist()

sick_test_df = sick_df[sick_df['SemEval_set'] == 'TEST']
sick_test_examples = list(zip(sick_test_df['sentence_A'].tolist(), sick_test_df['sentence_B'].tolist()))
sick_test_labels = sick_test_df['entailment_label'].tolist()

Check how many examples and labels you have got in each partition:

In [67]:
assert(type(sick_train_examples[3])==tuple)
print(len(sick_train_examples),len(sick_train_labels),len(sick_test_examples),len(sick_test_labels))

4934 4934 4906 4906


#4.7. (4 points) Use a pretrained language model for sequence embedding and train a regression on top

We can rely here on Huggingface which provides many pretrained models in its ```transformers``` library.

In [68]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 41.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 23.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 91.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 68.7 MB/s eta 0:00:00


Here we can use a relatively small model called DistilBert

In [69]:
import torch
import transformers

In [70]:
tokenizer = transformers.AutoTokenizer.from_pretrained("distilbert-base-uncased",padding=True)

In [71]:
distilbert = transformers.AutoModel.from_pretrained("distilbert-base-uncased")

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_layer_norm.bias', 'vocab_projector.bias', 'vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_transform.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Tokeniser of the model does a lot of heavy lifting. It takes in a sentence or a pair of sentences, concatenates them, tokenizes, adds [CLS] token (id: 101) and separator(s) [SEP] token (id: 102) and returns a nensor with the list of token ids:

In [72]:
inputs = tokenizer(sick_df.sentence_A.tolist()[5],sick_df.sentence_B.tolist()[5], return_tensors="pt")

In [73]:
print(inputs)

{'input_ids': tensor([[  101,  2045,  2003,  2053,  2879,  2652, 19350,  1998,  2045,  2003,
          2053,  2158,  5629,   102,  1037,  2177,  1997,  4268,  2003,  2652,
          1999,  1037,  4220,  1998,  2019,  2214,  2158,  2003,  3061,  1999,
          1996,  4281,   102]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1]])}


Now we can pass the tensor output of the tokenizer through the model, getting its hidden states:

In [74]:
with torch.no_grad():

    output = distilbert(**inputs)

Above, ```torch.no_grad()``` guarantees that no gradients are computed in this code block. So the model's weights cannot be updated. This is what we want now: obtain sentence pair vectors from the model, without making any changes to the model itself.

You should use ```output.last_hidden_state``` that stores the last hidden layer. From that, you only need the first vector, which corresponds to the [CLS] token. It is the first in the sequence so has index 0. What is the size of that vector? Check!

In [75]:
print(output.last_hidden_state[0][0].size(dim=0))

768


It is the last vector of the [CLS] token that is normally used for sequence classification with BERT models. Train and test a logistic regression classifier on top of (frozen) DistilBERT embeddings for sentence pairs in SICK:

In [76]:
distilbert_embdim = distilbert.config.hidden_size

entailment_model = Regression(distilbert_embdim,3)

Initialize an optimizer and loss function:

In [77]:
logregoptimiser=torch.optim.Adam(entailment_model.parameters())
loss_function = torch.nn.CrossEntropyLoss()

Now prepare the data: convert ```sick_train_examples``` and ```sick_test_examples``` into lists of torch tesors (embeddings of the sentence pairs)

In [78]:
# tokenized_pairs_train = [tokenizer(example, truncation=True, padding=True, return_tensors="pt") for example in sick_train_examples]
# tokenized_pairs_test = [tokenizer(example, truncation=True, padding=True, return_tensors="pt") for example in sick_test_examples]

# input_ids_train = [pair['input_ids'] for pair in tokenized_pairs_train]
# input_ids_test = [pair['input_ids'] for pair in tokenized_pairs_test]

# attention_masks_train = [pair['attention_mask'] for pair in tokenized_pairs_train]
# attention_masks_test = [pair['attention_mask'] for pair in tokenized_pairs_test]

tokenized_pairs_train = tokenizer(sick_train_examples, truncation=True, padding=True)
tokenized_pairs_test = tokenizer(sick_test_examples, truncation=True, padding=True)

input_ids_train = torch.tensor(tokenized_pairs_train['input_ids'])
attention_masks_train = torch.tensor(tokenized_pairs_train['attention_mask'])

input_ids_test = torch.tensor(tokenized_pairs_test['input_ids'])
attention_masks_test = torch.tensor(tokenized_pairs_test['attention_mask'])

# made 2 version, doesn't work with either

Similarly, convert ```sick_train_labels``` and ```sick_test_labels``` into lists of indices:

In [79]:
label_list = ['NEUTRAL', 'ENTAILMENT', 'CONTRADICTION']
label_to_index = {label: index for index, label in enumerate(label_list)}

label_indices_train = [label_to_index[label] for label in sick_train_labels]
label_indices_test = [label_to_index[label] for label in sick_test_labels]

Finally, train for 20 epochs and test a regression model on DistilBERT embeddings to classify SICK with the three entailment labels. Print out train and test accuracy and loss every 5 epochs.

In [ ]:
# tried to code with only 100 examples but it doesn't work no matter what we try to do.
epochs = 20

embeddings = distilbert(input_ids_train[:100], attention_mask=attention_masks_train[:100]).last_hidden_state

for epoch in trange(epochs, desc='Training Epochs'):
    entailment_model.train()  # Set the model to training mode

    logregoptimiser.zero_grad()
    outputs = entailment_model(embeddings)

    loss = loss_function(torch.squeeze(outputs), torch.tensor(label_indices_train[:100]))
    loss.backward()
    logregoptimiser.step()


Training Epochs:   0%|          | 0/20 [00:00<?, ?it/s]

RuntimeError: ignored

#4.8 (3 points) Fine-tuning

Can you improve the performance of entailment classification even further?

_Fine-tune_ DistilBERT on the task: train and test a logistic regression classifier on top of DistilBERT embeddings for sentence pairs in SICK while updating the model weights. You can define a new class that combines DistilBERT and regression models into a single model. This combined model passes sentence pair input through DistilBERT and uses the output embedding of the CLS token as input to regression, which finally produces the output of the whole combined model.

The weights of the conbined model (i.e. both DistilBERT weights and regression weights) can then be updated by the optimizer.

Train the whole pipeline for 5 epochs using the train/test split as above.

For this last exercise, using a GPU is essential. Even with a GPU, the computation may take a few minutes.

In [ ]:
epochs = 5
# We couldn't start this one since we did not finish 4.7. It was a really difficult part.


**Extra Credit**

You may observe that the straightforward finetuning approach shows overfitting: it ends up doing significantly better on the training set than the test set. This is acceptable for the purpose of this assignment, but not ideal.

Can you take measures to alleviate the overfitting, i.e. make the test performance better while reducing the gap between training and test prerformance? If you completed the rest of the assignment, you can try achieving this for extra credt.

Congratulations! You have reached the end of Assignment 4.